In [266]:
import pandas as pd
import numpy as np
from datetime import datetime
import os


from pathlib import Path

Path('covid.db').touch()

In [267]:
import sqlite3
conn = sqlite3.connect('covid.db')
c = conn.cursor()




In [268]:
# create a dictionary with all three CSV's
covid_data = { "states" : { "file" : "Resources/us-states.csv",
                    "schema" : '''CREATE TABLE states (date text, state text, cases int, deaths int)'''},
         "prison" : { "file" : "Resources/covid_prison_cases.csv",
                    "schema" : '''CREATE TABLE prison (name text, total_staff_deaths int, total_prisoner_cases int, total_prisoner_deaths int, as_of_date text )'''},
         "deaths" : {"file" :"Resources/3_covid_tracking_project_historical_testing_numbers_and_covid_deaths_by_state.csv",
                               "schema" : '''CREATE TABLE deaths (stage text, date text, total_population int, cumulative_deaths int,deaths_increase int, last_update_et text)'''}
       }




In [269]:
#create a DataFrame for all three csv's

for k in covid_data:
    d = pd.read_csv(covid_data[k]["file"])
    covid_data[k]["df"] = d
    
states_df = covid_data["states"]["df"]
prison_df = covid_data["prison"]["df"]
deaths_df = covid_data["deaths"]["df"]



In [270]:
# Keep the columns that needed for the analysis
states_df = states_df.loc[:, ("date","state","cases","deaths")]
prison_df = prison_df.loc[:, ("name","total_prisoner_cases","total_prisoner_deaths","as_of_date")]
deaths_df = deaths_df.loc[:, ("state","date","cumulative_deaths")]

In [271]:
#rename columns 
states_df.rename(columns = {"date" : "Date","state":"State","cases":"Number of Cases", "deaths":"Number of Deaths"}, inplace = True)
prison_df.rename(columns = {"name" : "State","total_prisoner_cases":"Total Prisoner Cases","total_prisoner_deaths":"Total Prisoner Deaths", "as_of_date":"Date"}, inplace = True)
deaths_df.rename(columns = {"state" : "State","date":"Date","cumulative_deaths":"Cumulative Prisoner Deaths"}, inplace = True)

In [272]:
# drop the rows with missing data
clean_states_df = states_df.dropna(axis = 0, how = 'any')
clean_prison_df = prison_df.dropna(axis = 0, how = 'any')
clean_deaths_df = deaths_df.dropna(axis = 0, how = 'any')

In [273]:
# change the date format

clean_prison_df[('Date')] = pd.to_datetime(clean_prison_df[('Date')]).dt.strftime('%Y-%m-%d')
clean_states_df[('Date')] = pd.to_datetime(clean_states_df[('Date')]).dt.strftime('%Y-%m-%d')
clean_deaths_df[('Date')] = pd.to_datetime(clean_deaths_df[('Date')]).dt.strftime('%Y-%m-%d')
clean_prison_df

/Users/sarapatel/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sarapatel/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,State,Total Prisoner Cases,Total Prisoner Deaths,Date
0,Delaware,0.0,0.0,2020-03-24
1,Illinois,3.0,0.0,2020-03-25
2,Alabama,0.0,0.0,2020-03-26
3,Alaska,0.0,0.0,2020-03-26
4,Arizona,0.0,0.0,2020-03-26
...,...,...,...,...
2617,Indiana,3467.0,51.0,2021-03-17
2618,Alaska,2382.0,5.0,2021-03-18
2619,Montana,1129.0,6.0,2021-03-18
2620,New York,6170.0,34.0,2021-03-18


In [274]:
# create an index
clean_deaths_df.reset_index(inplace=True)
clean_states_df.reset_index(inplace=True)
clean_prison_df.reset_index(inplace=True)
clean_deaths_df


,index,State,Date,Cumulative Prisoner Deaths
0,0,Alaska,2020-03-06,0.0
1,1,Alaska,2020-03-07,0.0
2,2,Alaska,2020-03-08,0.0
3,3,Alaska,2020-03-09,0.0
4,4,Alaska,2020-03-10,0.0
...,...,...,...,...
18174,18990,Wyoming,2021-03-03,682.0
18175,18991,Wyoming,2021-03-04,682.0
18176,18992,Wyoming,2021-03-05,682.0
18177,18993,Wyoming,2021-03-06,682.0


In [32]:

#c.execute('''SELECT * FROM states JOIN deaths''').fetchall()

for k in covid_data:
    # c.execute(covid_data[k]["schema"])
    # d.to_sql(k, conn, if_exists='replace', index = True)

    
# c.execute('''SELECT * FROM deaths''').fetchall()




[(0,
  'Alaska',
  '2020-03-06 00:00:00',
  737438,
  8.0,
  0.010848369625649884,
  0,
  None,
  0,
  None,
  None,
  0.0,
  0.0,
  0,
  None,
  None,
  None,
  None),
 (1,
  'Alaska',
  '2020-03-07 00:00:00',
  737438,
  12.0,
  0.016272554438474827,
  4,
  None,
  0,
  None,
  None,
  0.0,
  0.0,
  0,
  None,
  None,
  None,
  None),
 (2,
  'Alaska',
  '2020-03-08 00:00:00',
  737438,
  14.0,
  0.018984646844887296,
  2,
  None,
  0,
  None,
  None,
  0.0,
  0.0,
  0,
  None,
  None,
  None,
  None),
 (3,
  'Alaska',
  '2020-03-09 00:00:00',
  737438,
  23.0,
  0.031189062673743426,
  9,
  None,
  0,
  None,
  None,
  0.0,
  0.0,
  0,
  None,
  None,
  None,
  None),
 (4,
  'Alaska',
  '2020-03-10 00:00:00',
  737438,
  23.0,
  0.031189062673743426,
  0,
  None,
  0,
  None,
  None,
  0.0,
  0.0,
  0,
  None,
  3.0,
  0.0,
  0.0),
 (5,
  'Alaska',
  '2020-03-11 00:00:00',
  737438,
  46.0,
  0.06237812534748685,
  23,
  None,
  0,
  None,
  None,
  0.0,
  0.0,
  0,
  None,
  6.33,
 

In [30]:
from sqlalchemy import create_engine, MetaData, Table, Column, ForeignKey
from sqlalchemy.ext.automap import automap_base
engine = create_engine('sqlite:///covid.db', echo=True)

metadata = MetaData()

metadata.reflect(engine, only=['states', 'prison', 'deaths'])

Base = automap_base(metadata=metadata)

Base.prepare()


# mapped classes are ready
States, Prison, Deaths = Base.classes.states, Base.classes.prison,Base.classes.deaths

2021-03-24 18:26:39,406 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-24 18:26:39,407 INFO sqlalchemy.engine.base.Engine ()
2021-03-24 18:26:39,408 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-24 18:26:39,409 INFO sqlalchemy.engine.base.Engine ()
2021-03-24 18:26:39,410 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-03-24 18:26:39,411 INFO sqlalchemy.engine.base.Engine ()
2021-03-24 18:26:39,413 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("states")
2021-03-24 18:26:39,414 INFO sqlalchemy.engine.base.Engine ()
2021-03-24 18:26:39,415 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-03-24 18:26:39,416 INFO sqlalchemy.engine.base.Engine ('states',)
2021-03-24 18:26:39,417 INFO sqlalchemy.en

AttributeError: states

In [25]:
Base.classes.states

AttributeError: states